# Multilabel BERT Experiments

In this notebook we do some first experiments with BERT: we finetune a BERT model+classifier on each of our datasets separately and compute the accuracy of the resulting classifier on the test data.

For these experiments we use the `pytorch_transformers` package. It contains a variety of neural network architectures for transfer learning and pretrained models, including BERT and XLNET.

Two different BERT models are relevant for our experiments: 

- BERT-base-uncased: a relatively small BERT model that should already give reasonable results,
- BERT-large-uncased: a larger model for real state-of-the-art results.

In [1]:
PREFIX = "voting_so_automl"
BERT_MODEL = 'bert-large-uncased'
BATCH_SIZE = 16 if "base" in BERT_MODEL else 2
GRADIENT_ACCUMULATION_STEPS = 1 if "base" in BERT_MODEL else 8
MAX_SEQ_LENGTH = 100

## Data

We use the same data as for all our previous experiments. Here we load the training, development and test data for a particular prompt.

In [2]:
import sys
sys.path.append('../')

import ndjson
import glob
import numpy as np
from collections import Counter

from quillnlp.models.bert.preprocessing import preprocess, create_label_vocabulary

data_file = f"../data/interim/{PREFIX}_withprompt.ndjson"

with open(data_file) as i:
    data = ndjson.load(i)
    
label2idx = create_label_vocabulary(data)
idx2label = {v:k for k,v in label2idx.items()}
target_names = [idx2label[s] for s in range(len(idx2label))]

data_items = preprocess(data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH)
data_items = np.array(data_items)
    
labels = Counter()
for item in data:
    labels.update(item["label"])
print(labels)

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passi

Counter({'More Representative': 371, 'Miscellaneous': 223, 'Increased Voting': 218, 'Leaders': 109, 'Civic Duty': 25, 'Apathy': 10, 'Feels Represented': 10})


Read synthetic data, when available.

In [3]:
from collections import defaultdict

synth_files = glob.glob(f"../data/interim/{PREFIX}_withprompt_*.ndjson")

synth_data = []
for synth_file in synth_files:
    with open(synth_file) as i:
        synth_data.extend(ndjson.load(i))

preprocessed_synth_data = preprocess(synth_data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH)
        
synth_map = defaultdict(list)
for item, preprocessed_item in zip(synth_data, preprocessed_synth_data):
    synth_map[item["source_text"]].append(preprocessed_item)

I1108 11:34:58.040598 139696871913280 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-vocab.txt from cache at /home/yves/.cache/torch/transformers/9b3c03a36e83b13d5ba95ac965c9f9074a99e14340c523ab405703179e79fc46.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


## Model

We load the pretrained model and put it on a GPU if one is available. We also put the model in "training" mode, so that we can correctly update its internal parameters on the basis of our data sets.

In [4]:
import torch

from quillnlp.models.bert.train import train, evaluate
from quillnlp.models.bert.models import get_multilabel_bert_classifier

from quillnlp.models.bert.preprocessing import get_data_loader
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=1)
all_correct, all_predicted = [], []
all_test_data = []
for train_idx, test_idx in kf.split(data_items):

    train_and_dev_data = data_items[train_idx]
    cutoff = int(len(train_and_dev_data)/6*5)
    train_data = train_and_dev_data[:cutoff]
    dev_data = train_and_dev_data[cutoff:]
    test_data = data_items[test_idx]
    
    print("Train size:", len(train_data))
    
    synth_data = []
    for item in train_data:
        synth_data.extend(synth_map.get(item.text, []))
                
    train_dataloader = get_data_loader(np.concatenate((train_data, synth_data)), BATCH_SIZE)
    dev_dataloader = get_data_loader(dev_data, BATCH_SIZE)
    test_dataloader = get_data_loader(test_data, BATCH_SIZE, shuffle=False)

    print("Final train dataloader length:", len(train_dataloader))
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = get_multilabel_bert_classifier(BERT_MODEL, len(label2idx), device=device)
    output_model_file = train(model, train_dataloader, dev_dataloader, BATCH_SIZE, GRADIENT_ACCUMULATION_STEPS, device)
    
    print("Loading model from", output_model_file)
    device="cpu"

    model = get_multilabel_bert_classifier(BERT_MODEL, len(label2idx), model_file=output_model_file, device=device)
    model.eval()
    
    _, _, test_correct, test_predicted = evaluate(model, test_dataloader, device)
    all_correct.extend(test_correct)
    all_predicted.extend(test_predicted)
    all_test_data.extend(test_data)

Train size: 531
Final train dataloader length: 2458


I1108 11:35:00.358827 139696871913280 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /home/yves/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.4c88e2dec8f8b017f319f6db2b157fee632c0860d9422e4851bd0d6999f9ce38
I1108 11:35:00.361253 139696871913280 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 7,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1108 11:35:00.8875


Loss history: []
Dev loss: 0.19100835401978758


Epoch:   5%|▌         | 1/20 [04:37<1:27:52, 277.51s/it]


Loss history: [0.19100835401978758]
Dev loss: 0.19037974313660352


...


Loss history: [0.23329125689687552, 0.17894791580598662, 0.17482026204190873, 0.20566614400767894, 0.1897273830189887]
Dev loss: 0.20088162927681374





Epoch:  35%|███▌      | 7/20 [32:34<1:00:24, 278.84s/it]


Loss history: [0.23329125689687552, 0.17894791580598662, 0.17482026204190873, 0.20566614400767894, 0.1897273830189887, 0.20088162927681374]
Dev loss: 0.240225103545796



Loss history: [0.23329125689687552, 0.17894791580598662, 0.17482026204190873, 0.20566614400767894, 0.1897273830189887, 0.20088162927681374, 0.240225103545796]
Dev loss: 0.24361922593649338
No improvement on development set. Finish training.
Loading model from /tmp/model.bin


I1108 14:37:57.471852 139696871913280 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /home/yves/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.4c88e2dec8f8b017f319f6db2b157fee632c0860d9422e4851bd0d6999f9ce38
I1108 14:37:57.473618 139696871913280 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 7,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1108 14:37:57.9872

## Evaluation

In [5]:
correct = 0
at_least_one = 0
fp, fn, tp, tn = 0, 0, 0, 0
for c, p in zip(all_correct, all_predicted):
    if sum(c == p) == len(c):
        correct +=1
    
    for ci, pi in zip(c, p):
        if pi == 1 and ci == 1:
            at_least_one += 1
            break
    
    for ci, pi in zip(c, p):
        if pi == 1 and ci == 1:
            tp += 1
            same = 1
        elif pi == 1 and ci == 0:
            fp += 1
        elif pi == 0 and ci == 1:
            fn += 1
        else:
            tn += 1
            same =1
            
precision = tp/(tp+fp)
recall = tp/(tp+fn)
print("Data size:", len(all_predicted))
print("P:", tp, "/", tp+fp, "=", precision)
print("R:", tp, "/", tp+fn, "=", recall)
print("F:", 2*precision*recall/(precision+recall))
print("A:", correct/len(all_correct))
print("AL1:", at_least_one/len(all_correct))

Data size: 798
P: 802 / 968 = 0.8285123966942148
R: 802 / 966 = 0.8302277432712215
F: 0.8293691830403308
A: 0.7393483709273183
AL1: 0.8784461152882206


In [6]:
scores = {}
for item, predicted, correct in zip(all_test_data, all_predicted, all_correct):
    correct_labels = [idx2label[i] for i, l in enumerate(correct) if l == 1]
    predicted_labels = [idx2label[i] for i, l in enumerate(predicted) if l == 1]
    print("{}#{}#{}".format(item.text, ";".join(correct_labels), ";".join(predicted_labels)))    
        
    for label in predicted_labels + correct_labels:
        if label not in scores:
            scores[label] = {"tp": 0, "fp": 0, "fn": 0, "support": 0}
    
    for label in predicted_labels:
        if label in correct_labels:
            scores[label]["tp"] += 1
        else:
            scores[label]["fp"] += 1

    for label in correct_labels:
        scores[label]["support"] += 1
        if label not in predicted_labels:
            scores[label]["fn"] += 1
            
for label in scores:
    lp = scores[label]["tp"] / (scores[label]["tp"] + scores[label]["fp"]) if scores[label]["tp"] + scores[label]["fp"] > 0 else 0
    lr = scores[label]["tp"] / (scores[label]["tp"] + scores[label]["fn"]) if scores[label]["tp"] + scores[label]["fn"] > 0 else 0
    lf = 2*lp*lr/(lp+lr) if lp+lr > 0 else 0
    
    print(label, lp, lr, lf, scores[label]["support"])

Governments should make voting compulsory so we vote the correct person.#Leaders#Leaders
Governments should make voting compulsory so we should vote compulsory.#Miscellaneous#Miscellaneous
Governments should make voting compulsory so we can ensure we achieve the outcome we want as a society.#Miscellaneous#Miscellaneous
Governments should make voting compulsory so we are quick to make a decision.#Miscellaneous#Miscellaneous
Governments should make voting compulsory so voting say that it is a violation of personal liberties and choices.#Miscellaneous#Miscellaneous
Governments should make voting compulsory so voter turnout is mandatory#Miscellaneous#Increased Voting
Governments should make voting compulsory so voter participation will increase and the majority of the population will be represented in the outcome of a political election.#More Representative;Leaders#More Representative;Increased Voting
Governments should make voting compulsory so to encourage voters participation#Increased 

In [7]:
print(len([p for p in all_predicted if sum(p) == 0]), "/", len(all_predicted))

8 / 798
